# Nowcasting Hospitalizations
> Based on participatin in [German NowcastHub](covid19nowcasthub.de)

This notebook provides an extension to the authors participation in the [German Nowcasthub](covid19nowcasthub.de), a collaborative effort to nowcast the 7-day number of hospitailsations in Germany. See the site for further details. 

We quickly describe the model we used in this Hub:

## Method for model in NowcatsHub

To predict the number of hospitalisations we consider the reporting process of both reported COVID-19 cases and reported hospitalisations.
Recall that the reporting date of a COVID-19 case is shared for both the case and its hospitalisation, i.e. the case and hospitalisation are linked through this date.

As hospitalisations are only available as $7$-day rolling sums we use $7$-day rolling sums for daily reported incidences as well. 
To avoid dealing with the double weekday effect of both reporting date of the case and reporting date of the hospitalisation we divide the future hospitalisations we wish to predict into chunks of one week, which gets rid of the weekday effect for the hospitalisations.
This is depicted in the figure below.
Our prediction of each of these weekly chunks then consists of the fraction of hospitalisations of reported cases in the past.

![](reptri.png)

> Decomposition of the daily reported hospitalisation incidences into the <span style="color:rgb(255,121,0)">known incidences</span> $\color{#FF7900}H_{t,d}$, i.e. the **reporting triangle**, and <span style="color:rgb(0,116,122)">the future weekly increments</span> $\color{#00747A} H_{t, d + 7 (k + 1)} - H_{t, d + 7 k}$. <span style="color:rgb(0,51,89)">The last increment</span> might not be a weekly one, but we expect few cases to occur for such long delays.

More formally, denote by $h_{t,d}$ the number of hospitalisations with reporting date $t$ that are known $d$ days later. Unfortunately we only observe $$H_{t,d} = \sum_{s = t - 6}^{t} h_{s,d + (t - s)},$$ i.e. a weekly sum of reported hospitalisations.
On day $T$ our goal is to predict $H_{t,D}$ for large delays $D$ and days $t \leq T$, for which it clearly suffices to predict $H_{t, D} - H_{t, T - t}$ and add the known $H_{t, T - t}$ to this prediction. 
We rewrite this into a weekly telescoping sum

$$
H_{t,D} - H_{t,d} = \left(H_{t, d + 7} - H_{t,d}\right) + \left(H_{t, d + 14} - H_{t, d + 7}\right) + \dots + \left(H_{t,D} - H_{t, d + 7 K}\right),
$$

where $K = \lfloor (D -d) / 7 \rfloor$, reducing the task at hand to predict hospitalisations in the $k$-th week ahead, $H_{t, d + 7k} - H_{t, d + 7\cdot(k - 1)}$ for $k = 1, \dots, K$.

To leverage known reported incidences, rewrite this as 

$$
\frac{H_{t, d + 7k} - H_{t, d + 7\cdot(k - 1)}}{I_{t,d}}  I_{t, d} = p_{t,d,k}  I_{t,d}
$$

where $I_{t,d}$ is the $7$-day case incidence with reporting date $t$ known at time $t + d$, i.e. the incidenct case analouge of $H_{t,d}$.

Assuming that the proportions $p_{t,d,k}$ change slowly over time $t$ we estimate them by 

$$
\widehat {p_{t,d,k}} = \frac{H_{t - 7k, d + 7k} - H_{t - 7k, d + 7\cdot(k - 1)}}{I_{t - 7k,d}} = p_{t - 7k,d,k}
$$

and finally predict

$$
\widehat{H_{t,D}} = H_{t,d} + \left(\widehat{p_{t,d,1}} + \dots + \widehat{p_{t,d,K}}\right)  I_{t,d}.
$$

In essence, this model is a regression of reported hospitalisations on reported cases.

As hospitalisation is affected by age, we perform this procedure for all available age groups separately and finally aggregate over all age groups to obtain a nowcast for all age groups combined. 

This describes our point nowcast for $7$-day hospitalisations. 
To obtain uncertainty intervals we fit a normal (age groups 00-04 and 05-14) or lognormal (all other age groups) distribution to the past performance of our model. 
We chose these distributions based on explorative analysis and believe that these should be seen as heuristics rather than as a matter of fact, which is in line with the philosophy of our model to be as simple as possible.

Denote by $\hat H_{t,D,s}$ the nowcast made for date $t$ on date $s \geq t$. Starting with date $t + D$ the definite $H_{t,D}$ is known and we can estimate the absolute prediction error $\varepsilon_{t,s} = H_{t,D} - \hat H_{t,D,s}$ and the relative prediction error $\eta_{t,s} = \log \left( H_{t,D} - H_{t, s - t}\right) - \log \left( \hat H_{t,D,s} - H_{t, s- t} \right)$.
For the nowcast for date $t$ made on date $s$ we estimate the standard deviation $\hat\sigma$ of $\varepsilon_{t - D - i, s - D - i}$ or $\eta_{t - D - i, s - D - i}$ (age groups 00-04, 05-14 and others respectively), $i = 0, \dots, 27$ by its empirical counterpart.
The estimated predictive distribution which informs our prediction intervals is then $\mathcal N (\hat H_{t,D,s}, \hat\sigma^2)$ (age groups 00-04 and 05-14) or $\mathcal{LN} \left( \log \left(\hat H_{t,D,s} - H_{t, s - t}\right), \hat\sigma^2 \right) + H_{t, s - t}$ (all other age groups).

In this document, we want to extend this model to account for temporal dependencies, i.e. model $p_{t,d,k}, I_{t,d}$ and $H_{t,d}$ by a state space model. 
